In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score

sns.set()
pd.set_option('display.precision', 3)

import warnings
warnings.filterwarnings('ignore')

In [7]:
# Leemos del csv
df_train = pd.read_csv("./sample_data/train.csv")
df_test = pd.read_csv("./sample_data/test.csv")

In [9]:
df_train.head()

,Characteristics.LotFeatures,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.features_reso.results,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,...,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.Heating,Structure.LivingArea,Structure.NewConstructionYN,Structure.ParkingFeatures,Structure.Rooms.RoomsTotal,Structure.YearBuilt,Tax.Zoning,UnitTypes.UnitTypeType
0,NaN,3200.0,NaN,NaN,NaN,NaN,NaN,"['Cooling.CeilingFans', 'Flooring.Carpet', 'Fl...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,0.0,commr,NaN
1,NaN,NaN,NaN,3.8,3.7,3.8,3.8,"['Appliances.GasRange', 'Appliances.Range', 'A...",NaN,4.2,...,NaN,NaN,['natural gas'],3175.0,False,['off alley'],6.0,NaN,NaN,NaN
2,NaN,18750.0,NaN,NaN,NaN,NaN,NaN,"['Flooring.Carpet', 'ParkingFeatures.Garage', ...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,1926.0,commr,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['CommunityFeatures.Lake', 'WaterfrontFeatures...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,agric,NaN
4,"['horses allowed', 'paddock', 'pond(s)']",NaN,NaN,3.6,NaN,NaN,3.6,"['Appliances.Refrigerator', 'Appliances.Microw...",NaN,4.2,...,NaN,NaN,['other'],0.0,False,"['unassigned', 'off street']",4.0,2006.0,NaN,NaN


In [10]:
df_test.head()

,Characteristics.LotFeatures,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.features_reso.results,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,...,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.Heating,Structure.LivingArea,Structure.NewConstructionYN,Structure.ParkingFeatures,Structure.Rooms.RoomsTotal,Structure.YearBuilt,Tax.Zoning,UnitTypes.UnitTypeType
0,NaN,8640.0,2.0,3.3,2.2,2.0,2.3,"['Appliances.Dishwasher', 'Appliances.Microwav...",2.8,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1899.0,commr,NaN
1,NaN,13937.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0,commr,NaN
2,NaN,NaN,3.3,3.1,3.1,2.9,3.1,"['Appliances.GasRange', 'Appliances.IceMaker',...",3.4,3.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,singl,NaN
3,NaN,2500.0,NaN,NaN,NaN,NaN,NaN,"['Appliances.Cooktop', 'Appliances.GasCooktop'...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,offic,NaN
4,"['fenced yard', 'golf course lot', 'landscaped']",NaN,3.9,3.6,4.0,4.0,3.9,"['Appliances.GasWaterHeater', 'Appliances.Refr...",3.9,3.7,...,4.0,5.0,"['natural gas', 'forced air']",8960.0,False,NaN,17.0,2008.0,NaN,NaN
